1. Parse out all macaca mulatta entries

In [1]:
!grep -B16 'Macaca mulatta' MHC.dat.txt > macaca_mulatta_listOfIDs.mhc.txt

In [2]:
!grep 'ID   ' macaca_mulatta_listOfIDs.mhc.txt | sed 's/ID[[:space:]]*//g' > macaca_mulatta_listOfIDs.parsed.mhc.txt

In [17]:
import time
def importEntries(f):
    pList = []
    with open(f) as fOpen:
        currentItem = []
        for i in fOpen:
            i = i.rstrip('\r\n')
            if i[0:3] == '//':
                pList.append(currentItem)
                currentItem = []
            else:
                currentItem.append(i)
    return pList

def parseOutSelectedEntries(l, itmList):
    rList = []
    for itm in l:
        checkItem = itm[0]
        splitItem = checkItem.split(' ')
        if len(splitItem) == 1:
            splitItem = checkItem.split('\t') # try tab instead of whitespace
        checkItemFilteredList = list(filter(lambda x: x != '', splitItem))
        if checkItemFilteredList[1] in itmList:
            rList.append(itm)
    return rList

idList = []
with open('macaca_mulatta_listOfIDs.parsed.mhc.txt') as f:
    for i in f:
        i = i.rstrip('\n\r')
        idList.append(i)
importedList = importEntries('MHC.dat.txt')
parsedAndImportedList = parseOutSelectedEntries(importedList, idList)
parsedFileName = 'macaca_mulatta.parsed.mhc.txt'
for itm in parsedAndImportedList:
    o = '\n'.join(itm)
    o += '\n//\n'
    # print(o)
    with open(parsedFileName, 'a') as fWrite:
        fWrite.write(o)

2. import data from parsed text file

In [29]:
def parsedAndCastRange(xList):
    # requires: string in the format '1..5', returns [1,5] as list of ints
    unparsedRangeAsList = xList.split('.')
    parsedRange = list(filter(lambda x: x!= '', unparsedRangeAsList))
    return [int(parsedRange[0]), int(parsedRange[1])]

def parsedExonOutput(s):
    retList = []
    for i in s:
        listOfValues = i
        exonSeq = i[-1]
        exon_ct = []
        exon_ct_list = []
        mxLen = len(listOfValues[3])
        for j in listOfValues[3]:
            exon_ct.append(j[0])
            exon_ct_list.append((j[1][0], j[1][1]))
        # indexTransition = 0
        indexTransition = 0
        checkForIntron = 0
        for x in range(0, mxLen, 1):
            exonStartAndStop = exon_ct_list[x]
            exonStart = exonStartAndStop[0]
            if x == 0:
                exonStart = exonStartAndStop[0] - 1
            exonStop = exonStartAndStop[1] + 1
            exonSequence = exonSeq[exonStart:exonStop]
            indexTransition = exonStartAndStop[1]+1
            r = str(listOfValues[0]) + ':' + str(exon_ct[x]) + ',' + exonSequence
            retList.append(r)
    return retList

alleleList = []
with open(parsedFileName, 'r') as f:
    s_id = ''
    s_alleleID = ''
    s_exonList = []
    alleleExonRange = []
    alleleExonSeq = []
    exonSeqs = ''
    exonCt = 1
    seqFlag = False
    for i in f:
        i = i.rstrip('\r\n')
        sLine = i.split(' ')
        sLineList = list(filter(lambda x: x != '', sLine))
        if sLineList[0] == 'KW':
            s_id = sLineList[1]
        elif sLineList[0] == 'ID':
            s_alleleID = sLineList[1]
        elif sLineList[0] == 'FT':
            if sLineList[1] == 'allele':
                alleleExonRange = parsedAndCastRange(sLineList[2])
            elif sLineList[1] == 'exon':
                exonSeqRange = parsedAndCastRange(sLineList[2])
                exonN = 'exon' + str(exonCt)
                alleleExonSeq.append((exonN, exonSeqRange))
                exonCt += 1
            else:
                continue
        elif sLineList[0] == 'SQ':
            seqFlag = True
            continue
        else:
            if seqFlag:
                if sLineList[0] == '//':
                    '''modify this as necessary.  It is unknown why python string indices
                    are completely wacky--possibly something Bash does?
                    be advised you may need to change the range from 0:3 
                    or change the string range in the parsing above to 0:2'''
                    seqFlag = False
                    alleleList.append([s_id, s_alleleID, alleleExonRange, alleleExonSeq, exonSeqs])
                    alleleExonSeq = []
                    exonCt = 1
                    exonSeqs = ''
                    continue
                else:
                    s = ''.join(sLineList[:-1])
                    exonSeqs += s
            else:
                continue


[['Mamu-AG*01:01', 'NHP00355', [1, 1073], [('exon1', [1, 49]), ('exon2', [50, 319]), ('exon3', [320, 595]), ('exon4', [596, 871]), ('exon5', [872, 988]), ('exon6', [989, 1021]), ('exon7', [1022, 1068]), ('exon8', [1069, 1073])], 'CTCCTCCTAGTGCTCTCAGGGGTCCTGGCCCCGACCCAGACCCGGGCGGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCTGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGAGAGTCCGAGGATGGAGCCGCGGGCGCCGTGGGTGGAGCAGGAGGGGCCGGAGTATTGGGACCGGGAGACACAGAACATGAAGACCCCGACACAGACCTACCGAGTGAACCTGCGGACCCTGCTCCGCTACTACAACCAGAGCGAGGCCGGGTCTCACACCATCCAGAAGATGTACGGCTGCGACCTGGGGCCCGACGGGCGCCTCCTCCGCGGGTATGAACAGTTCGCCTACGACGGCAGGGATTACATCATCCTGAACGAGGACCTGCGCTCCTGGACTGCCGCGGACATGGCGGCTCAGAACACCCAGCGCAAGTGGGAGGCGGCGGGTGCGGCGGAGCAGCACAGAACCTACCTGGAGGGCGAGTGCCTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCGCGGACCCCCCCAAGACACATGTGACCCACCACCCCGTCTCTGACTACGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGAGAGGAACAAACTCAGGACACAGAGCTCATGGAGACCAGGCCTACAGGGAATGGAACCTT

3. convert data structure to conform to expected format of

          [
            ('Allele1Name', ['exon1-string', 'exon2-string',...,'exonN-string']),
            ('Allele2Name', ['exon1-string', 'exon2-string',...,'exonN-string']),
            ('Allele3Name', ['exon1-string', 'exon2-string',...,'exonN-string'])
          ]



In [54]:
def reformatListOfTuples(l):
    # requires SINGLE list from list of lists
    retList = []
    for i in l[3]:
        cdsStart = int(i[1][0])-1
        cdsStop = int(i[1][1])
        cdsString = l[-1]
        cds = cdsString[cdsStart:cdsStop]
        retList.append(cds)
    return (l[0], retList)

convertedDataStructure = []
for i in alleleList:
    convertedDataStructure.append(reformatListOfTuples(i))       

[('Mamu-I*01:06:23', ['ATGCGGGTTATGGCGCCCGGAACCCTCCTCCTGCTGCTCTCGGGGGCCCTGGCCCTGACCGAGACCTGGGCCG', 'GCTCGCACTCCTTGAGGTATTTCGGCACCACAGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGAGAGTCCGAGGATGGAGCCGCGGGCGCCGTGGATGGAGCAGGAGGGGCCGGAGTATTGGGAAGAGGAGACACGGATCGCCAAGGCCCGCGCACAGACTGAGCGAGGGAACCTGCGGACCGCGCTCCGCTACTACAACCAGAGCGAGGGGG', 'GGTCTCACACCCTCCAGAAGATGTGTGGCTGCGACCTGGGGCCGGACGGGCGCCTCCTCCGCGGGTATTACCAGTCCGCCTACGACGGCAGGGATTACATCGCCCTGAATGAGGACCTGCGCTCCTGGACCGCTGCGGGTGAGGCGGCTCAGAACACCCAGCGCAAGTGGGAGGCGGCCGGTGAAGCAGAGCAGCGGAGAGCCTACCTGGAGGGCCGGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAAGGGGAAGGAGACGCTGCAGCGCGCGG', 'ATCCCCCAAAGACACACGTGACCCACCACCCCGTCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGAGCAAACTCAGGACACCGAGCTTGTGGAGACCAGGCCAGGAGGAGATGGAAACTTCCAGAAGTGGGCAGCTGTGGTGGTGCCTTCTGGAGAAGAGCAGAGATACACGTGCCATGTGCAGCATGAGGGGCTGCCGGAGCCCCTCACCCTGAGATGGG', 'AGCCATCTTCCCAGTCCACCATCCCCATCATGGGCATTGTTGCTGGCCTGGCTGTCCTAGCAGTTG

4. import data from Excel spreadsheet as [(AlleleID1, allele1-sequence),(AlleleID2, allele2-sequence),...]

In [126]:
def parseOutFromExcelTSV(f, l):
    retList = []
    stopAt = len(l)
    removeHeader = True
    with open(f, 'r') as fOpen:
        for i in fOpen:
            if removeHeader:
                removeHeader = False
                continue
            i = i.rstrip('\r\n')
            iSplit = i.split('\t')
            t = tuple([iSplit[x] for x in l])
            retList.append(t)
    return retList

allele_Excel_list = parseOutFromExcelTSV('22695_IPD_Sequences_based on_PacBio-SPADES_concordance.tsv', [3,9])

'GATGGCGATCATGGCGCCCCGAACCCTCCTCCTGGTGCTCTCAGGGGCCCTGGTCCTGACCCAGACCCGGGCGGGTGAGTGCGGGGTCGGGAGGGAAATGGCCTCTGCGGGGAGCAGCGAGGGGCCCGCCCGGCGGGGGCGCAGGACCCGGGGAGCCGCGCGGGGAGGAGGGTCGGGCGGGTCTCAGCCTCTCCTCGCCCCCAGGCTCTCACTCCATGAAGTATTTCTACACCTCCATGTCCCGGCCCGGCCGCGGGCAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAAAGGATGGAGCCGCGGGCGCCGTGGGTGGAGCAGGAGGGTCCGGAGTATTGGGACCGGGAGACACGGAACATGAAGACCGAGACACAGAATGCCCCAGTGAACCTGCGGACCCTGCTCCGCTACTACAACCAGAGCGAGGCCGGTGAGTGACCCCGGCCCGGGGCGCAGGTCACGACCCCTCCGCATCCCCCACGGACGGCCCGCGTCGCCCCGAGTCTCCGGGTCCCAGATCCAACCCGAAGCCTCGGGACCCGGGGACCCTTGACCCGGGAGAGCCCCAGGCGCCTTTACCCGGTTTCATTTTCAGTTGAGGCCAAAATCCCCGCGGGTTGGTCGGGGCGGGGCGAGGCTTGGTGGGCGGGGCTGACCGCGGGGGCGGGGCCAGGGTCTCACACCCTCCAGAGGATGGTTGGCTGCGACCTGGGGCCCGACGGGCGCCTCCTCCGCGGGTATGAACAGTACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCCTGGACCGCGGCGGACGTGGCGGCTCAGAACACCCAGCGCAAGTGGGAGGCGGCGGATGTGGCGGAGAGCATGAGAGCCTACCTGGAGGGCCAGTGCGTGGAGTGGCTCCCCAGATACCTGGAGAAGGGGAAGGAGACGCTGCAGCGCACGGGTACCA

5. Map both sets of sequences

In [32]:
def exonPosMatching(startPosStart, seq, exonString):
    startPos = seq.find(exonString)
    if startPos < startPosStart:
        return (startPos, -1)
    elif startPos > -1:
        endPos = startPos + len(exonString)
        return (startPos, endPos)
    else:
        return (-1, -1)

def matchExonsToString(alleleString, alleleName, alleleExonTuple, alleleRgx):
    res_pos = []
    _start_pos = 0
    filter_res = list(filter(lambda x: x[0] == alleleName, alleleExonTuple))
    if filter_res:
        _filter_res = filter_res[0][1][:-1]
        filter_res_lastExon = filter_res[0][1][-1]
        for i in _filter_res[0][1]:
            matched = exonPosMatching(_start_pos, alleleString, i)
            _start_pos = matched[0]
            res_pos.append(matched)
    else:
        string_scan = alleleName.split(alleleRgx)
        stopCt = len(string_scan) # + 1
        for i in range(1, stopCt):
            res = alleleRgx.join(string_scan[:-i])
            filter_res = list(filter(lambda x: x[0] == res, alleleExonTuple))
            if filter_res:
                _filter_res = filter_res[0][1][:-1]
                filter_res_lastExon = filter_res[0][1][-1]
                for x in _filter_res:
                    matched = exonPosMatching(_start_pos, s_seq, x)
                    _start_pos = matched[0]
                    res_pos.append(matched)
                break # will stop after first match
    if not res_pos:
        return None
    end_pos = _start_pos + len(filter_res[0][1][-2])
    lastExonPosSearch = s_seq[end_pos:]
    matched = exonPosMatching(0, lastExonPosSearch, filter_res_lastExon)
    if matched[1] > -1:
        matched1 = matched[0] + end_pos
        matched2 = matched[1] + end_pos
        matched = (matched1, matched2)
    res_pos.append(matched)
    checkPos = list(filter(lambda x: x[1] == -1, res_pos))
    if len(checkPos) > 1:
        return (0, alleleName, filter_res[0][0], [alleleString, filter_res[0][1], res_pos])
    else:
        return (1, alleleName, filter_res[0][0], [alleleString, filter_res[0][1], res_pos])




123
456


In [25]:
if args.outputType == 'exonList':
                  res = parsedExonOutput(alleleList)
                  for i in res:
                      print(i)
              else:
                  if args.outputType == 'mergedFastaList':
                      for i in alleleList:
                          hString = '>' + str(i[0]) + ' ' + str(i[1])
                          seqString = str(i[-1])
                          print(hString + '\n' + seqString)
                  else:
                      res = parsedExonOutput(alleleList)
                      for i in res:
                          iSplit = i.split(',')
                          hString = '>' + str(iSplit[0])
                          seqString = str(iSplit[1])
                          print(hString + '\n' + seqString)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)